# Session 1

In the `quickstart_tutorial`, we saw how to control the simulator using a Jupyter Notebook. Now, we will see how to make the agents move and implement basic behaviors.

For this session, we only need one agent in the simulation. To do so, the server can be started in the command line with the options `--n_agents 1` and `n_objects 0`.
<!-- TODO: change this with file system once done !-->
<!-- TODO: set objects to 0 !-->

First of all, we instantiate a `controller` (this will be done at the start of every notebook from now on)

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
import numpy as np
controller = NotebookController()

This time, only one agent should be present. We can check this with this instruction:

In [2]:
controller.agents

Next, we start the simulation:

In [ ]:
controller.run(threaded=True)

Exception in thread Thread-5 (_run):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.8/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/martial/vivarium/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/opt/homebrew/Cellar/python@3.11/3.11.8/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/martial/vivarium/vivarium/controllers/notebook_controller.py", line 125, in _run
    self.state = self.client.step()
                 ^^^^^^^^^^^^^^^^^^
  File "/Users/martial/vivarium/vivarium/simulator/grpc_server/simulator_client.py", line 55, in step
    self.state = proto_to_state(self.stub.Step(Empty()))
                                ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/martial/vivarium/.venv/lib/python

### Making the agents move

You should now know how to set the attributes of agents. For this time, as there is only one agent, we can create a shortcut for the next cells of code:

In [ ]:
agent = controller.agents[0]

Now, the variable `agent` refers to the first (and only) agent of the simulation. You can simply write `agent` instead of `controller.agents[0]` for the next instructions.

**Q1:** What is the condition for the agent to turn left? to turn right? to move straight forward? to stop?

*This is a cell where you can write text instead of code.* Double click on this text and enter your answer here. Once it is done, press `Shift-Enter`.

**Q2:** How can you make the agent move forward without any rotation? Write the corresponding code in the cell below. 

**Q3:** Now write the code making the agent stop:

**Q4:** And the code to make it move in a large circle:

Let's try more complex choregraphies. A useful function for this is the ability to wait for a given time by using `controller.wait(x)`, where `x` has to be replaced by the time to wait for, in seconds. Here is an example, where the agent goes forward during 3 seconds, then turns right during 0.5 second, then goes forward again during 2 seconds, and finally stops.

In [ ]:
# Move forward, by setting each wheel speed at the same positive value
agent.left_spd = 1.
agent.right_spd = 1.

# Wait for 3 seconds, keeping moving forward at the same speed
controller.wait(3)

# Turn right by setting a positive speed to the left wheel and stoping the right one (null speed)
agent.left_spd = 1.
agent.right_spd = 0.

# Keep turning for 0.5 second
controller.wait(0.5)

# Move forward again for 2 seconds
agent.left_spd = 1.
agent.right_spd = 1.
controller.wait(2)

# Stop the agent
agent.left_spd = 0.
agent.right_spd = 0.

A few important remarks regarding the code above:
- each line beginning with a `#` symbol corresponds to a comment and will therefore not be executed (it is just to explain verbally what the code does),
- when calling `controller.wait(.)`, the cell that is currently running will simply wait for the given time in seconds before executing the next line of code. this also means that you cannot execute anything else before the wait time is over.

Now, let's repeat the previous choreography 4 times. In the code below, the first line, `for i in range(4):`, means *repeat 4 times the indented code below* (actually it is a bit more complicated than this, but this out of the scope of the current session).  The indented code is the exact same as just before (only the comments have been removed). You can change the number `4` in the first line by any number `x`, to repeat it `x` times instead of 4.

In [ ]:
for i in range(4):
    agent.left_spd = 1.
    agent.right_spd = 1.
    controller.wait(3)
    agent.left_spd = 1.
    agent.right_spd = 0.
    controller.wait(0.5)
    agent.left_spd = 1.
    agent.right_spd = 1.
    controller.wait(2)
    agent.left_spd = 0.
    agent.right_spd = 0.

**Q5:** By executing the code just above, you will observe that the agent actually only stops at the very end of the choreography, but not between each run of the loop. Copy-paste the code of the last cell above in the cell below and add one line of code so that the agent will stop for 1 second at each run of the loop.

**Q6:** Write the code allowing the agent to move roughly in a "8" shape. To do so you'll have to fine tune the waiting times by trial on error. Then make the agent repeat it 6 times.

### Defining agent's behaviors

We can define the behavior of an agent as a Python function taking as argument an `Agent` and returning the values of the left and right motors, in this order. Within the body of the function, one can access all the agents attribute. Usually, defining a behavior requires to access the value of the agent sensors. This done through the `agent.left_prox` and `agent.right_prox` attributes, that return the value of the left and right sensors, respectively.

Let's define four behaviors, here corresponding to the four canonical behaviors of [Braitenberg Vehicles](https://en.wikipedia.org/wiki/Braitenberg_vehicle):

In [ ]:
def aggression(agent):
    return agent.right_prox, agent.left_prox

def fear(agent):
    return agent.left_prox, agent.right_prox

def love(agent):
    return 1. - agent.left_prox, 1. - agent.right_prox

def shy(agent):
    return 1. - agent.right_prox, 1. - agent.left_prox

The code above only declares the behaviors, now we need to attach them to the agents in the simulator. Let's attach the `shy` behavior to the first five agents and set their color to blue ; and the `aggression` behavior to the four last agents and set their color to red:

In [ ]:
for ag in controller.agents[:5]:
    ag.color = 'blue'
    ag.detach_all_behaviors()
    ag.attach_behavior(shy)
for ag in controller.agents[5:]:
    ag.color = 'red'
    ag.detach_all_behaviors()
    ag.attach_behavior(aggression)

All agents are now equipped with a behavior. We can launch the simulation with the code below ; then stop it with the code on the next line.

In [ ]:
controller.run(threaded=True)

In [ ]:
controller.stop()